# Mettre à disposition un modèle par le biais d’une API

Lino Galiana  
2025-06-14

<div class="badge-container"><div class="badge-text">Pour essayer les exemples présents dans ce tutoriel :</div><a href="https://github.com/linogaliana/python-datascientist-notebooks/blob/main/notebooks/modelisation/7_mlapi.ipynb" target="_blank" rel="noopener"><img src="https://img.shields.io/static/v1?logo=github&label=&message=View%20on%20GitHub&color=181717" alt="View on GitHub"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/vscode-python?autoLaunch=true&name=«7_mlapi»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-vscode.sh»&init.personalInitArgs=«modelisation%207_mlapi%20correction»" target="_blank" rel="noopener"><img src="https://custom-icon-badges.demolab.com/badge/SSP%20Cloud-Lancer_avec_VSCode-blue?logo=vsc&logoColor=white" alt="Onyxia"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/jupyter-python?autoLaunch=true&name=«7_mlapi»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-jupyter.sh»&init.personalInitArgs=«modelisation%207_mlapi%20correction»" target="_blank" rel="noopener"><img src="https://img.shields.io/badge/SSP%20Cloud-Lancer_avec_Jupyter-orange?logo=Jupyter&logoColor=orange" alt="Onyxia"></a>
<a href="https://colab.research.google.com/github/linogaliana/python-datascientist-notebooks-colab//blob/main//notebooks/modelisation/7_mlapi.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a><br></div>

Ce chapitre présente la deuxième application
d’une journée de cours que j’ai
donné à l’Université Dauphine dans le cadre
des *PSL Data Week*.

L’objectif de ce chapitre est d’amener à développer
une API du type de [celle-ci](https://dvf-simple-api.lab.sspcloud.fr).

Dérouler les *slides* associées ci-dessous ou [cliquer ici](https://linogaliana.github.io/dauphine-week-data/#/title-slide)
pour les afficher en plein écran.

<pre class="sourceCode yaml code-with-copy"><code class="sourceCode yaml"></code><button title="Copy to Clipboard" class="code-copy-button"><i class="bi"></i></button></pre>

<iframe class="sourceCode yaml code-with-copy" src="https://linogaliana.github.io/dauphine-week-data/#/title-slide">

</iframe>

Le chapitre précédent constituait une introduction à la création
de *pipelines* de *machine learning*.
Ce chapitre va aller plus loin en montrant la démarche pour le rendre
disponible à plus grande échelle par le biais d’une API pouvant
être consommée avec de nouvelles données. L’objectif de celle-ci est
de ne pas contraindre les réutilisateurs d’un modèle
à disposer d’un environnement technique complexe
pour pouvoir utiliser le même modèle que celui entraîné précédemment.

# 1. Exemple de réutilisation d’un modèle sous forme d’API

Un exemple d’API obtenue à l’issue de ce chapitre est
mis à disposition sur <https://dvf-simple-api.lab.sspcloud.fr/>.
La documentation de l’API est disponible [ici](https://dvf-simple-api.lab.sspcloud.fr/docs).

Cette API est utilisable dans plusieurs langages.

En `Python`, par exemple, cela donnera:

In [1]:
import requests

pieces_principales = 6
surface = 50
url = f"https://dvf-simple-api.lab.sspcloud.fr/predict?month=4&nombre_lots=1&code_type_local=2&nombre_pieces_principales={pieces_principales}&surface={surface}"
requests.get(url).json()

728358.5461884077

Néanmoins, l’un des intérêts de proposer
une API est que les utilisateurs du modèle
ne sont pas obligés d’être des pythonistes.
Cela accroît grandement la cible des ré-utilisateurs
potentiels.

Cette approche ouvre notamment la possibilité de
faire des applications interactives qui utilisent,
en arrière plan, notre modèle entraîné avec `Python`.

# 2. Etape 1: créer une application en local

Mettre en place une API consiste à gravir une marche
dans l’échelle de la reproductibilité par rapport
à fournir un *notebook*. Ces derniers
ne sont pas les outils les plus adaptés
pour partager autre chose que du code, à faire tourner
de son côté.

Il est donc naturel de sortir des *notebooks*
lorsqu’on commence à aller vers ce niveau de mise à
disposition.
Par le biais de
scripts `Python` lancés en ligne de commande,
construits en exportant le code du chapitre précédent
de nos notebooks, on pourra
créer une base de départ propre.

Il est plus naturel de privilégier une interface de développement
généraliste comme VSCode à Jupyter lorsqu’on franchit
ce rubicon. L’exercice suivant permettra donc
de créer cette première application minimale, à
exécuter en ligne de commande.

In [2]:
from IPython.display import HTML
style = '''

    <style>
    .callout {
    border: 2px solid #d1d5db;
    border-radius: 8px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    margin-bottom: 20px;
    background-color: #ffffff;
    padding: 15px;
}
.callout-header-note {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #47648a;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-tip {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #41745d;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-exercise {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #c46aad;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-warning {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #967b30;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-important {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #86252b;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-caution {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #a7663b;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}


.callout-body {
    margin: 10px 0;
}
    </style>
    
'''
content_html = '''

    <div class="callout callout-exercise">
        <div class="callout-header-exercise">
            <i class="fa-solid fa-pen-fancy"></i> Exercice 1: créer des scripts pour entraîner le modèle
        </div>
        <div class="callout-body">
            <p>Le dépôt <code>Github</code> qui permet de construire l\'API <em>from scratch</em>
est <a href="https://github.com/linogaliana/api-dvf">disponible ici</a>.
Nous allons emprunter quelques éléments, par-ci par-là,
pour faire notre application en local. </p>
<ul>
<li>Créer un nouveau service <code>VSCode</code> sur le <code>SSPCloud</code> en paramétrant dans l\'onglet
<code>Networking</code> le port 5000 ;</li>
<li>Utiliser la commande suivante depuis le terminal:</li>
</ul>
<p><code>shell
mkdir app
cd app</code></p>
<p>Depuis le menu des fichiers, créer quatre fichiers dont le contenu
suit:</p>
<ul>
<li><code>requirements.txt</code>: récupérer le contenu sur <a href="https://raw.githubusercontent.com/linogaliana/api-dvf/main/requirements.txt">cette page</a> ;</li>
<li><code>getdvf.py</code>: récupérer le contenu sur <a href="https://raw.githubusercontent.com/linogaliana/api-dvf/main/getdvf.py">cette page</a> ;</li>
<li><code>train.py</code>: récupérer le contenu sur <a href="https://raw.githubusercontent.com/linogaliana/api-dvf/main/train.py">cette page</a> ;</li>
<li>
<p><code>api.py</code>: récupérer le contenu sur <a href="https://raw.githubusercontent.com/linogaliana/api-dvf/main/main.py">cette page</a>.</p>
</li>
<li>
<p>Exécuter <code>getdvf.py</code> puis <code>train.py</code> pour stocker en local le modèle entraîné</p>
</li>
<li>Ajouter <code>model.joblib</code> au <code>.gitignore</code> <em>(si vous utilisez <code>Git</code>)</em></li>
<li>Créer un script <code>test.py</code> qui contient la fonction suivante et la teste après avoir importé votre modèle (<code>load(\'pipe.joblib\')</code> en n\'oubliant pas <code>from joblib import load</code>):</li>
</ul>
        </div>
    </div>
    
'''
HTML(f'<script src="https://kit.fontawesome.com/3c27c932d3.js" crossorigin="anonymous"></script>\n{style}\n{content_html}')

Exercice 1: créer des scripts pour entraîner le modèle
 
 
 Le dépôt Github qui permet de construire l'API from scratch 
est disponible ici .
Nous allons emprunter quelques éléments, par-ci par-là,
pour faire notre application en local. 
 
 Créer un nouveau service VSCode sur le SSPCloud en paramétrant dans l'onglet
 Networking le port 5000 ; 
 Utiliser la commande suivante depuis le terminal: 
 
 shell
mkdir app
cd app 
 Depuis le menu des fichiers, créer quatre fichiers dont le contenu
suit: 
 
 requirements.txt : récupérer le contenu sur cette page ; 
 getdvf.py : récupérer le contenu sur cette page ; 
 train.py : récupérer le contenu sur cette page ; 
 
 api.py : récupérer le contenu sur cette page . 
 
 
 Exécuter getdvf.py puis train.py pour stocker en local le modèle entraîné 
 
 Ajouter model.joblib au .gitignore (si vous utilisez Git ) 
 Créer un script test.py qui contient la fonction suivante et la teste après avoir importé votre modèle ( load('pipe.joblib') en n'oubliant pas from joblib import load ):

In [3]:
import pandas as pd

def predict(
    month: int = 3,
    nombre_lots: int = 1,
    code_type_local: int = 2,
    nombre_pieces_principales: int = 3,
    surface: float = 75
) -> float:
    """
    """

    df = pd.DataFrame(
        {
            "month": [month],
            "Nombre_de_lots": [nombre_lots],
            "Code_type_local": [code_type_local],
            "Nombre_pieces_principales": [nombre_pieces_principales],
            "surface": [surface]
        }
    )

    prediction = model.predict(df)

    return prediction

# 3. Etape 2: créer une API en local

Le script précédent constitue déjà un progrès dans
la reproductibilité. Il rend plus facile le réentraînement
d’un modèle sur le même jeu de données. Néanmoins,
il reste tributaire du fait que la personne désirant
utiliser du modèle utilise `Python` et sache réentrainer
le modèle dans les mêmes conditions que vous.

Avec `FastAPI`, nous allons très facilement pouvoir
transformer cette application `Python` en une API.

In [4]:
from IPython.display import HTML
style = '''

    <style>
    .callout {
    border: 2px solid #d1d5db;
    border-radius: 8px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    margin-bottom: 20px;
    background-color: #ffffff;
    padding: 15px;
}
.callout-header-note {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #47648a;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-tip {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #41745d;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-exercise {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #c46aad;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-warning {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #967b30;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-important {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #86252b;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}

.callout-header-caution {
    font-weight: bold;
    margin-bottom: 10px;
    color: #ffffff;
    background-color: #a7663b;
    padding: 10px;
    border-radius: 6px 6px 0 0;
}


.callout-body {
    margin: 10px 0;
}
    </style>
    
'''
content_html = '''

    <div class="callout callout-exercise">
        <div class="callout-header-exercise">
            <i class="fa-solid fa-pen-fancy"></i> Exercice 2: créer des scripts pour entraîner le modèle
        </div>
        <div class="callout-body">
            <ul>
<li>La ligne ci-dessous du script <code>api.py</code> récupère un modèle pré-entraîné enregistré sur un espace de stockage</li>
</ul>
<p>~~~python
download_file("https://minio.lab.sspcloud.fr/projet-formation/diffusion/python-datascientist/pipe.joblib", \'pipe.joblib\')
~~~</p>
<p>Retirer cette ligne de votre script, pour utiliser
le modèle que vous venez d\'entraîner. </p>
<ul>
<li>Déployer en local l\'API avec la commande</li>
</ul>
<p><code>shell
uvicorn api:app --reload --host "0.0.0.0" --port 5000</code></p>
<ul>
<li>A partir du <code>README</code> du <a href="https://datalab.sspcloud.fr/my-services">service VSCode</a>,
se rendre sur l\'URL de déploiement, 
ajouter <code>/docs/</code> à celui-ci et observer la documentation de l\'API </li>
<li>Se servir de la documentation pour tester les requêtes <code>/predict</code></li>
<li>Récupérer l\'URL d\'une des requêtes proposées. La tester dans le navigateur
et depuis <code>Python</code> avec <code>Requests</code> (<code>requests.get(url).json()</code>)</li>
<li>Optionnel: faire tourner le même code dans un autre environnement que le SSPCloud (par exemple une installation de <code>Python</code> en local) pour voir que ça fonctionne de manière identique.</li>
</ul>
        </div>
    </div>
    
'''
HTML(f'<script src="https://kit.fontawesome.com/3c27c932d3.js" crossorigin="anonymous"></script>\n{style}\n{content_html}')

Exercice 2: créer des scripts pour entraîner le modèle
 
 
 
 La ligne ci-dessous du script api.py récupère un modèle pré-entraîné enregistré sur un espace de stockage 
 
 ~~~python
download_file("https://minio.lab.sspcloud.fr/projet-formation/diffusion/python-datascientist/pipe.joblib", 'pipe.joblib')
~~~ 
 Retirer cette ligne de votre script, pour utiliser
le modèle que vous venez d'entraîner. 
 
 Déployer en local l'API avec la commande 
 
 shell
uvicorn api:app --reload --host "0.0.0.0" --port 5000 
 
 A partir du README du service VSCode ,
se rendre sur l'URL de déploiement, 
ajouter /docs/ à celui-ci et observer la documentation de l'API 
 Se servir de la documentation pour tester les requêtes /predict 
 Récupérer l'URL d'une des requêtes proposées. La tester dans le navigateur
et depuis Python avec Requests ( requests.get(url).json() ) 
 Optionnel: faire tourner le même code dans un autre environnement que le SSPCloud (par exemple une installation de Python en local) pour voir que ça fonctionne de manière identique.

# 4. Aller plus loin: mettre à disposition cette API de manière pérenne

L’étape précédente permettait de créer un point d’accès
à votre modèle depuis n’importe quel type de client. A chaque
requête de l’API, le script `api.py` était exécuté et
renvoyait son *output*.

Ceci est déjà un saut de géant dans l’échelle de la
reproductibilité. Néanmoins, cela reste artisanal: si votre
serveur local connait un problème (par exemple, vous *killez* l’application), les clients ne recevront plus de réponse,
sans comprendre pourquoi.

Il est donc plus fiable de mettre en production sur des
serveurs dédiés, qui tournent 24h/24 et qui peuvent
également se répartir la charge de travail s’il y a
beaucoup de demandes instantanées.

Ceci dépasse néanmoins
le cadre de ce cours et sera l’objet
d’un cours dédié en 3e année de l’ENSAE: [“Mise en production de projets *data science*”](https://ensae-reproductibilite.github.io/website/) donné par Romain Avouac et moi.